In [69]:
'''Felix Andersson, Janine de Vries, DV2626'''

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer


In [124]:
# Steg 1: Ladda dataset
exercise_data = pd.read_csv('gym.csv', delimiter=';')
important_data = exercise_data[['Main_muscle', 'Target_Muscles', 'Synergist_Muscles', 'Stabilizer_Muscles', 'Antagonist_Muscles', 'Dynamic_Stabilizer_Muscles', 'Difficulty (1-5)', 'Secondary Muscles']]
important_data.head()

,Main_muscle,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Difficulty (1-5),Secondary Muscles
0,Shoulder,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
1,Shoulder,"Anterior Deltoid,","Pectoralis Major, Clavicular, Triceps Brachii,...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head, Biceps Brachii, Short Head,",4,"Lateral Deltoid, Triceps Brachii"
2,Shoulder,"Anterior Deltoid,","Deltoid, Lateral, Supraspinatus, Triceps Brach...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head,",3,"Lateral Deltoid, Triceps Brachii"
3,Shoulder,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
4,Shoulder,"Anterior Deltoid,","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"


In [ ]:


# Handle NaN values by replacing them with empty strings
important_data["Target_Muscles"] = important_data["Target_Muscles"].fillna("")

# Split multi-value strings into lists
important_data["Target_Muscles"] = important_data["Target_Muscles"].apply(lambda x: x.split(", ") if x else [])

# Use MultiLabelBinarizer to encode
mlb = MultiLabelBinarizer()
binary_matrix = mlb.fit_transform(important_data["Target_Muscles"])

# Add the binary encoded lists as a new column
important_data["Target_Muscles"] = binary_matrix.tolist()

# Save the modified important_data to a new CSV file


# Optional: Print muscle names (classes) and check the modified important_data
print("Muscle names:", mlb.classes_)
important_data.head()


Muscle names: ['Anterior Deltoid,' 'Biceps Brachii,' 'Brachialis,' 'Brachioradialis,'
 'Gastrocnemius,' 'Gluteus Maximus' 'Gluteus Maximus,' 'Hamstrings'
 'Hamstrings,' 'Hip Abductors,' 'Hip Adductors,' 'Hip Flexors'
 'Infraspinatus' 'Lateral Deltoid,' 'Latissimus Dorsi' 'Latissimus Dorsi,'
 'Levator Scapulae' 'Lower Trapezius' 'Middle Trapezius,'
 'Pectoralis Major Clavicular,' 'Pectoralis Major Sternal'
 'Pectoralis Major Sternal,' 'Posterior Deltoid,' 'Pronators,'
 'Quadriceps,' 'Rhomboids' 'Rhomboids,' 'Serratus Anterior,' 'Soleus,'
 'Subscapularis,' 'Supinator,' 'Supraspinatus,' 'Teres Major'
 'Teres Minor,' 'Tibialis Anterior,' 'Triceps Brachii,' 'Upper Trapezius'
 'Wrist Extensors,' 'Wrist Flexors,']


C:\Users\felea\AppData\Local\Temp\ipykernel_12768\3590505401.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_data["Target_Muscles"] = important_data["Target_Muscles"].fillna("")
C:\Users\felea\AppData\Local\Temp\ipykernel_12768\3590505401.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  important_data["Target_Muscles"] = important_data["Target_Muscles"].apply(lambda x: x.split(", ") if x else [])
C:\Users\felea\AppData\Local\Temp\ipykernel_12768\3590505401.py:16: SettingWithCopyWarning: 
A 

,Main_muscle,Target_Muscles,Synergist_Muscles,Stabilizer_Muscles,Antagonist_Muscles,Dynamic_Stabilizer_Muscles,Difficulty (1-5),Secondary Muscles
0,Shoulder,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
1,Shoulder,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Pectoralis Major, Clavicular, Triceps Brachii,...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head, Biceps Brachii, Short Head,",4,"Lateral Deltoid, Triceps Brachii"
2,Shoulder,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Deltoid, Lateral, Supraspinatus, Triceps Brach...","Trapezius, Upper, Levator Scapulae,",NaN,"Triceps, Long Head,",3,"Lateral Deltoid, Triceps Brachii"
3,Shoulder,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"
4,Shoulder,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","Pectoralis Major, Clavicular, Deltoid, Lateral...","Trapezius, Upper, Levator Scapulae, Wrist Exte...",NaN,NaN,2,"Anterior Deltoid, Serratus Anterior"


In [ ]:
important_data.loc[:, "Target_Muscles"] = important_data["Target_Muscles"].fillna("")

# Split multi-value strings into lists
important_data.loc[:, "Target_Muscles"] = important_data["Target_Muscles"].apply(lambda x: x.split(", ") if x else [])

# Use MultiLabelBinarizer to encode
mlb = MultiLabelBinarizer()
binary_matrix = mlb.fit_transform(important_data["Target_Muscles"])

# Replace the original "Target_Muscles" column with the encoded binary data
important_data.loc[:, "Encoded_Target_Muscles"] = binary_matrix.tolist()
important_data.head()


ValueError: Must have equal len keys and value when setting with an ndarray